In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split as TTS,  GridSearchCV  
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB as NB


import nltk
from nltk.corpus import stopwords, sentiwordnet, wordnet
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

import spacy

from typing import List
from pprint import pprint


import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim

import tqdm

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping


import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
lotr_train = pd.read_csv('lotr_train.csv')
lotr_test = pd.read_csv('lotr_test.csv')

imp_char = ["FRODO", "SAM", "GANDALF", "PIPPIN", "MERRY", "GOLLUM", "GIMLI", "THEODEN", "FARAMIR", "SAURON", "ARAGORN", "SMEAGOL"]

In [11]:
nlp = spacy.load("en_core_web_sm")

def word2vec_df(dialogs):
    # Extracting norm vector values
    word_vectors = []
    for dialogue in dialogs:
        tokens = nlp(dialogue)
        dialogue_vectors = [token.vector_norm for token in tokens]
        word_vectors.append(dialogue_vectors)

    # Padding 
    max_length = max(len(dialogue) for dialogue in word_vectors)
    for i in range(len(word_vectors)):
        word_vectors[i] += [np.nan] * (max_length - len(word_vectors[i]))

    df = pd.DataFrame(word_vectors)
    df.columns = [f"word_{i}" for i in range(1, max_length + 1)]
    return df  

train_w2v = word2vec_df(lotr_train['dialog'])
test_w2v = word2vec_df(lotr_test['dialog'])

In [12]:
train_w2v

,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,word_10,...,word_148,word_149,word_150,word_151,word_152,word_153,word_154,word_155,word_156,word_157
0,7.765202,9.092802,8.749047,9.259231,8.784856,9.041077,8.425842,8.957423,13.258844,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.936516,11.274338,12.535010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.807209,7.082729,10.126812,9.273765,7.891686,8.183380,10.324145,7.371899,9.027041,7.104752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.937764,7.750165,8.915595,8.662035,6.803154,6.460621,11.343762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10.943976,7.265589,10.792604,13.414934,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,11.657317,9.785878,7.969328,7.751654,7.139582,10.619580,13.255728,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1668,9.113812,8.394229,6.745904,9.888923,8.690373,10.508968,11.182974,7.502326,6.799454,8.679674,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1669,12.096506,6.420546,8.593643,9.325191,8.675666,8.009504,8.922383,10.044628,13.553076,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1670,9.798004,8.581070,9.354257,13.826755,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
train_w2v.to_csv('train_w2v.csv', index=False)
test_w2v.to_csv('test_w2v.csv', index=False)